In [15]:
import pandas as pd
import numpy as np
import math
import heapq
import time
import folium
import ipywidgets as widgets
from IPython.display import display

In [16]:
# -------------------------
# 1. Load Cleaned Cities CSV
# -------------------------
cities_df = pd.read_csv("Data/Cities_of_SriLanka.csv")

# Convert to dictionary for easy lookup
city_coords = {row['name_en']: (row['latitude'], row['longitude']) for _, row in cities_df.iterrows()}


In [17]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

In [18]:
# -------------------------
# 3. Build k-Nearest Graph
# -------------------------

K = 10  # number of neighbors
cities = list(city_coords.keys())
graph = {city: [] for city in cities}

for city in cities:
    lat1, lon1 = city_coords[city]
    distances = []
    for other in cities:
        if city == other:
            continue
        lat2, lon2 = city_coords[other]
        d = haversine(lat1, lon1, lat2, lon2)
        distances.append((d, other))
    distances.sort()
    for d, neighbor in distances[:K]:
        graph[city].append((neighbor, d))

In [19]:
# Build helper structures from the graph produced in cell 4
# `graph` expected as: { city: [(neighbor, distance), ...], ... }
nearest_cities = {}
graph_dist = {}

for city in cities:
    neighbors = []
    dist_map = {}
    for item in graph.get(city, []):
        # Support multiple possible item shapes robustly
        if isinstance(item, (list, tuple)):
            if len(item) >= 2:
                nb, d = item[0], item[1]
            elif len(item) == 1:
                nb, d = item[0], None
            else:
                continue
        else:
            nb, d = item, None

        neighbors.append(nb)
        if d is not None:
            dist_map[nb] = d

    nearest_cities[city] = neighbors
    graph_dist[city] = dist_map

# Display nearest cities for a sample
print("\nNearest Cities for Sample Cities:")
for city in cities[:20]:
    print(f"{city}: {nearest_cities.get(city, [])}")

# Create and display a subset distance matrix (km)
subset_names = cities[:20]
subset_matrix = pd.DataFrame(index=subset_names, columns=subset_names, dtype=float)
for city in subset_names:
    for nb in subset_names:
        if nb in graph_dist.get(city, {}):
            subset_matrix.at[city, nb] = round(graph_dist[city][nb], 2)

print("\nSubset Distance Matrix (km):")
display(subset_matrix)


Nearest Cities for Sample Cities:
Akkaraipattu: ['Navatkadu', 'Addalaichenai', 'Oluvil', 'Thambiluvil', 'Nintavur', 'Tirukovil', 'Irakkamam', 'Samanthurai', 'Karativu', 'Sainthamaruthu']
Ambagahawatta: ['Ampara', 'Deegawapiya', 'Devalahinda', 'Marathamune', 'Pannalagama', 'Wanagamuwa', 'Uhana', 'Hingurana', 'Paragahakele', 'Digamadulla Weeragoda']
Ampara: ['Ambagahawatta', 'Deegawapiya', 'Devalahinda', 'Marathamune', 'Pannalagama', 'Wanagamuwa', 'Uhana', 'Hingurana', 'Paragahakele', 'Digamadulla Weeragoda']
Bakmitiyawa: ['Hulannuge', 'Wadinagala', 'Kandaudapanguwa', 'Madawalalanda', 'Lahugala', 'Siyambalanduwa', 'Komari', 'Buddama', 'Tirukovil', 'Thambiluvil']
Deegawapiya: ['Ambagahawatta', 'Ampara', 'Devalahinda', 'Marathamune', 'Pannalagama', 'Wanagamuwa', 'Uhana', 'Hingurana', 'Paragahakele', 'Digamadulla Weeragoda']
Devalahinda: ['Ambagahawatta', 'Ampara', 'Deegawapiya', 'Marathamune', 'Pannalagama', 'Wanagamuwa', 'Uhana', 'Hingurana', 'Paragahakele', 'Digamadulla Weeragoda']
Diga

,Akkaraipattu,Ambagahawatta,Ampara,Bakmitiyawa,Deegawapiya,Devalahinda,Digamadulla Weeragoda,Dorakumbura,Gonagolla,Hulannuge,Kalmunai,Kannakipuram,Karativu,Kekirihena,Koknahara,Kolamanthalawa,Komari,Lahugala,Irakkamam,Mahaoya
Akkaraipattu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.69,NaN,NaN,NaN,NaN,NaN,12.28,NaN
Ambagahawatta,NaN,NaN,0.00,NaN,0.00,0.00,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ampara,NaN,0.00,NaN,NaN,0.00,0.00,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bakmitiyawa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.60,NaN,NaN,NaN,NaN,NaN,NaN,20.93,18.13,NaN,NaN
Deegawapiya,NaN,0.00,0.00,NaN,NaN,0.00,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Devalahinda,NaN,0.00,0.00,NaN,0.00,NaN,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Digamadulla Weeragoda,NaN,10.12,10.12,NaN,10.12,10.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dorakumbura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.93,NaN,7.58,NaN,NaN,NaN,20.09
Gonagolla,NaN,19.17,19.17,NaN,19.17,19.17,15.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hulannuge,NaN,NaN,NaN,11.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.78,NaN,NaN


In [20]:
# -------------------------
# 4. Shortest Path Algorithms
# -------------------------

def dijkstra(graph, start, end):
    pq = [(0, start, [])]
    visited = set()
    while pq:
        (dist, node, path) = heapq.heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        path = path + [node]
        if node == end:
            return dist, path
        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                heapq.heappush(pq, (dist+weight, neighbor, path))
    return float("inf"), []


def bellman_ford(graph, start, end):
    dist = {node: float("inf") for node in graph}
    pred = {node: None for node in graph}
    dist[start] = 0
    
    for _ in range(len(graph)-1):
        for u in graph:
            for v, w in graph[u]:
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    pred[v] = u
    
    # Reconstruct path
    path = []
    node = end
    while node:
        path.insert(0, node)
        node = pred[node]
    return dist[end], path

# A* Algorithm
def a_star(graph, start, end):
    def heuristic(u, v):
        lat1, lon1 = city_coords[u]
        lat2, lon2 = city_coords[v]
        return haversine(lat1, lon1, lat2, lon2)
    
    open_set = [(0, start, [])]
    g_score = {node: float("inf") for node in graph}
    g_score[start] = 0
    f_score = {node: float("inf") for node in graph}
    f_score[start] = heuristic(start, end)
    visited = set()
    
    while open_set:
        (f, node, path) = heapq.heappop(open_set)
        if node in visited:
            continue
        visited.add(node)
        path = path + [node]
        if node == end:
            return g_score[end], path
        for neighbor, weight in graph[node]:
            tentative_g = g_score[node] + weight
            if tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, end)
                heapq.heappush(open_set, (f_score[neighbor], neighbor, path))
    return float("inf"), []


In [21]:
source_dd = widgets.Combobox(
    placeholder='Select source city',
    options=sorted(list(city_coords.keys())),
    description='Source City:',
    ensure_option=True,
    disabled=False
)

destination_dd = widgets.Combobox(
    placeholder='Select destination city',
    options=sorted(list(city_coords.keys())),
    description='Destination City:',
    ensure_option=True,
    disabled=False
)

# Display the dropdowns
print("Select the source and destination cities:")
display(source_dd, destination_dd)

Select the source and destination cities:


Combobox(value='', description='Source City:', ensure_option=True, options=('Aandankulam', 'Adampan', 'Addalai…

Combobox(value='', description='Destination City:', ensure_option=True, options=('Aandankulam', 'Adampan', 'Ad…

In [22]:
run_button = widgets.Button(description="Find Shortest Path")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        source = source_dd.value
        destination = destination_dd.value
        if source not in city_coords or destination not in city_coords:
            print("Please select valid cities from the dropdowns.")
            return
        
        print(f"Finding shortest path from {source} to {destination}\n")

        # Dijkstra
        start_time = time.time()
        dist, path = dijkstra(graph, source, destination)
        elapsed = round(time.time()-start_time, 6)
        print(f"Dijkstra -> Distance: {round(dist,2)} km | Time: {elapsed} s")

        # Bellman-Ford
        start_time = time.time()
        dist, path = bellman_ford(graph, source, destination)
        elapsed = round(time.time()-start_time, 6)
        print(f"Bellman-Ford -> Distance: {round(dist,2)} km | Time: {elapsed} s")

        # A* algorithm
        start_time = time.time()
        dist, path = a_star(graph, source, destination)
        elapsed = round(time.time()-start_time, 6)
        print(f"A* Algorithm -> Distance: {round(dist,2)} km | Time: {elapsed} s")

        

        # -------------------------
        # 6. Folium Visualization
        # -------------------------
        
        if path:
            lat, lon = city_coords[source]
            m = folium.Map(location=[lat, lon], zoom_start=8)

            # Add cities to map
            for city in path:
                lat, lon = city_coords[city]
                folium.Marker([lat, lon], popup=city, icon=folium.Icon(color="blue")).add_to(m)

            # Draw path line
            coords = [(city_coords[city][0], city_coords[city][1]) for city in path]
            folium.PolyLine(coords, color="red", weight=3, opacity=0.8).add_to(m)

            display(m)   # ⬅️ Shows map inline in Jupyter

run_button.on_click(on_button_clicked)

display(run_button, output)

Button(description='Find Shortest Path', style=ButtonStyle())

Output()